# AIMO3 Model Training - Fine-Tuning for Tool-Integrated Reasoning (TIR)

This notebook implements the training pipeline to fine-tune `Qwen2.5-32B-Instruct` (or `DeepSeek-R1-Distill`) on the `NuminaMath-TIR` dataset.

**Goal**: Teach the model to *interleave* Python code execution with reasoning.

### Technology Stack
*   **Library**: `Unsloth` (2x faster, 70% less memory) - Critical for fine-tuning 32B models on consumer/Kaggle GPUs.
*   **Dataset**: `AI-MO/NuminaMath-TIR`
*   **Technique**: QLoRA (4-bit quantization)

In [ ]:
# 1. Install Unsloth and Dependencies
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
import torch
from unsloth import FastLanguageModel

# 2. Configuration
max_seq_length = 2048 # Can increase to 4096 if VRAM allows
dtype = None # None for auto detection. Float16 for Tesla T4, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. 4bit is highly recommended.

# 3. Load Base Model
# We use Qwen2.5-Math-7B or 32B (if you have A100/H100)
# For free Colab/Kaggle T4s, stick to 7B. For competition H100, use 32B.
model_name = "unsloth/Qwen2.5-Math-7B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
# 4. Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Data Preparation
We need to format `NuminaMath-TIR` into the format the model expects.
The format should encourage the model to output Python code.

In [ ]:
from datasets import load_dataset

# 5. Load Dataset
dataset = load_dataset("AI-MO/NuminaMath-TIR", split = "train")

# 6. Formatting Function
prompt_template = """User: Solve the following math problem. Use Python code if necessary.
Problem: {}

Assistant: {}"""

def formatting_prompts_func(examples):
    problems = examples["problem"]
    solutions = examples["solution"]
    texts = []
    for problem, solution in zip(problems, solutions):
        # Numina dataset solutions usually already contain markdown code blocks
        text = prompt_template.format(problem, solution) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 7. Training Arguments
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set to -1 for full epoch, or higher for real training (e.g., 1000)
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
# 8. Train the Model
trainer_stats = trainer.train()

In [ ]:
# 9. Save the Model (LoRA Adapters)
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

# To push to Hugging Face Hub (Private or Public)
# model.push_to_hub("your_username/aimo3-qwen-tir", token = "...")
# tokenizer.push_to_hub("your_username/aimo3-qwen-tir", token = "...")

In [ ]:
# 10. Inference Test
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    "User: Solve the following math problem. Use Python code if necessary.\nProblem: Calculate the factorial of 5.\n\nAssistant:"
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)